In [63]:
class Peak(object):
    def __init__(self,id,mz,rt,intensity,level):
        self.id = id
        self.mz = mz
        self.rt = rt
        self.level = level
        self.intensity = intensity
        self.children = []
        self.parent = None

In [64]:
ms1name = '/Users/simon/Dropbox/MS2LDA Manuscript Sections/Matrices/Beer3pos_MS1filter_Method3_ms1.csv'
ms2name = '/Users/simon/Dropbox/MS2LDA Manuscript Sections/Matrices/Beer3pos_MS1filter_Method3_ms2.csv'
ms1peaks = {}
ms2peaks = []
with open(ms1name,'r') as f:
	heads = f.readline()
	for line in f:
		sline = line.split(',')
		ms1peaks[str(sline[1])] = Peak(int(sline[1]),float(sline[5]),float(sline[4]),float(sline[6]),int(sline[3]))

with open(ms2name,'r') as f:
	heads = f.readline()
	for line in f:
		sline = line.split(',')
		parentid = str(sline[2])
		parent = ms1peaks[parentid]
		newpeak = Peak(int(sline[1]),float(sline[5]),float(sline[4]),float(sline[6]),int(sline[3]))
		parent.children.append(newpeak)
		newpeak.parent = parent
		ms2peaks.append(newpeak)






In [65]:
print "Loaded {} MS1 and {} MS2 peaks".format(len(ms1peaks),len(ms2peaks))

Loaded 1422 MS1 and 27128 MS2 peaks


In [66]:
import numpy as np
def mass_hit(mz1,mz2,tol):
    if 1e6*np.abs(mz1-mz2)/mz1 < tol:
        return True
    else:
        return False

def rt_hit(rt1,rt2,tol):
    if np.abs(rt1-rt2) < tol:
        return True
    else:
        return False
    
to_find = [(314.14,269.69,'both')]
for mz,rt,prefix in to_find:
    hits = []
    for pid in ms1peaks:
        if mass_hit(mz,ms1peaks[pid].mz,5) and rt_hit(rt,ms1peaks[pid].rt,0.1):
            hits.append((pid,np.abs(mz-ms1peaks[pid].mz)))
            if len(hits) > 1:
                print "ARGH"
            else:
                fname = prefix + "_" + str(ms1peaks[pid].mz) + "_" + str(ms1peaks[pid].rt) + ".txt"
                with open(fname,'w') as f:
                    for child in ms1peaks[pid].children:
                        f.write("" + str(child.mz) + "," + str(child.intensity) + "\n")
                
    